# Backtest Avancé Morningstar (Optimisé)

In [ ]:
import sys
sys.path.append('../..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import tensorflow as tf
from Morningstar.model.architecture import MorningstarTradingModel
from Morningstar.workflows.morningstar_trading import MorningstarTradingWorkflow
from Morningstar.utils.bt_analysis import get_metrics, backtest_analysis
from Morningstar.utils.plot_analysis import plot_equity_vs_asset

# Configuration
config = {
    "pairs": ["BTC/USDT", "ETH/USDT"],
    "timeframe": "1h",
    "initial_wallet": 10000,
    "leverage": 3,
    "stop_loss": 0.3,
    "take_profit": 0.6,
    "maker_fee": 0.0002,
    "taker_fee": 0.0006
}

# Chargement des données et du modèle
data = {pair: pd.read_parquet(f'data/{pair.replace("/", "-")}_1h.parquet') 
       for pair in config["pairs"]}

model = MorningstarTradingModel(input_shape=(60, 19), num_classes=3)
model.model = tf.keras.models.load_model('morningstar_model.h5')

# Initialisation du backtest
trader = MorningstarTradingWorkflow(
    model=model,
    pairs=config["pairs"],
    initial_capital=config["initial_wallet"],
    leverage=config["leverage"]
)

# Exécution du backtest
results = trader.run_advanced_backtest(
    data=data,
    stop_loss=config["stop_loss"],
    take_profit=config["take_profit"],
    maker_fee=config["maker_fee"],
    taker_fee=config["taker_fee"]
)

# Analyse des résultats
metrics = get_metrics(results["trades"], results["days"])
display(HTML(f"""
<h2>Résultats du Backtest</h2>
<table>
<tr><td>Profit Total:</td><td>{metrics['profit']:.2f}%</td></tr>
<tr><td>Sharpe Ratio:</td><td>{metrics['sharpe']:.2f}</td></tr>
<tr><td>Win Rate:</td><td>{metrics['win_rate']:.2f}%</td></tr>
</table>
"""))

# Visualisations complètes
plt.figure(figsize=(14, 10))

# 1. Performance du capital
plt.subplot(2, 2, 1)
plt.plot(results['days']['wallet'], label='Capital')
plt.title('Évolution du capital')
plt.legend()

# 2. Drawdown
plt.subplot(2, 2, 2)
plt.plot(results['days']['drawdown'], color='red')
plt.title('Drawdown')

# 3. Répartition des trades
plt.subplot(2, 2, 3)
results['trades']['result'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Répartition des trades')

# 4. Profit par trade
plt.subplot(2, 2, 4)
results['trades']['profit'].hist(bins=50)
plt.title('Distribution des profits')

plt.tight_layout()
plt.show()

# Analyse détaillée
print("\n=== ANALYSE DÉTAILLÉE ===")
print(f"Nombre total de trades: {len(results['trades'])}")
print(f"Ratio profit/perte: {metrics['profit_factor']:.2f}")
print(f"Plus gros gain: {results['trades']['profit'].max():.2f}%")
print(f"Plus grosse perte: {results['trades']['profit'].min():.2f}%")

# Export des résultats
results["trades"].to_csv("morningstar_backtest_results.csv")
results["days"].to_csv("morningstar_daily_results.csv")
print("\nRésultats exportés:")
print("- morningstar_backtest_results.csv (détails des trades)")
print("- morningstar_daily_results.csv (performance journalière)")

# Analyse comparative
print("\n=== COMPARAISON AVEC HOLD ===")
hold_return = (data['BTC/USDT'].iloc[-1]['close'] - data['BTC/USDT'].iloc[0]['close']) / data['BTC/USDT'].iloc[0]['close'] * 100
print(f"Retour stratégie: {metrics['profit']:.2f}%")
print(f"Retour HODL: {hold_return:.2f}%")
print(f"Alpha généré: {metrics['profit'] - hold_return:.2f}%")

# Analyse temporelle
print("\n=== PERFORMANCE TEMPORELLE ===")
monthly_perf = results['days'].groupby(pd.to_datetime(results['days']['day']).dt.to_period('M'))['wallet'].last().pct_change() * 100
print("Performance mensuelle:\n", monthly_perf.dropna().to_string())

# Heatmap des performances
plt.figure(figsize=(12, 4))
monthly_perf.unstack().plot(kind='bar', stacked=True)
plt.title('Performance mensuelle')
plt.ylabel('Rendement %')
plt.show()

# Analyse des trades par paire
print("\n=== PERFORMANCE PAR PAIRE ===")
pair_perf = results['trades'].groupby('pair')['profit'].agg(['mean', 'count', 'sum'])
print(pair_perf.sort_values('sum', ascending=False).to_string())

# Sauvegarde du rapport complet
with open('morningstar_backtest_report.txt', 'w') as f:
    f.write(f"Rapport de Backtest - {datetime.now().strftime('%Y-%m-%d')}\n\n")
    f.write(f"Period: {data['BTC/USDT'].index[0].date()} to {data['BTC/USDT'].index[-1].date()}\n")
    f.write(f"Final Wallet: {results['days']['wallet'].iloc[-1]:.2f} USD\n")
    f.write(f"Total Return: {metrics['profit']:.2f}%\n")
    f.write(f"Sharpe Ratio: {metrics['sharpe']:.2f}\n")
    f.write(f"Max Drawdown: {metrics['max_drawdown']:.2f}%\n")
    f.write(f"\n=== Détails par paire ===\n{pair_perf.to_string()}")
print("\nRapport complet sauvegardé dans morningstar_backtest_report.txt")